# Kinetodynamic model of Omnibot

### Imports and prparations

In [ ]:
from sympy import init_printing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from math import modf

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, BlockMatrix)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
init_vprinting()

![Esquema de la rueda del Omnibot](images_dejabot/rueda.png)

![Alt text](images_dejabot/sist_body.png)

## Kinematics

In [ ]:
t, r, d, s = symbols('t r d s')
alpha, beta, sigma, psi, theta= dynamicsymbols('alpha beta sigma psi theta')
alpha, beta, sigma, psi, theta, t, r, d, s

In [ ]:
v_j = Matrix([sigma*cos(alpha), sigma*sin(alpha),0])
om_wheel = Matrix([0, theta.diff(t), psi.diff(t)])
jc = Matrix([0,0,r])

v_j, om_wheel, jc

In [ ]:
v_b_prim = v_j + om_wheel.cross(jc)
v_b_prim

### 2D
Keeping only the x y components:

In [ ]:
W = Matrix([
    [r, cos(alpha)],
    [0, sin(alpha)]
])
T = Matrix([
    [cos(beta), -sin(beta)],
    [sin(beta), cos(beta)]
])

W,T

In [ ]:
A = Matrix([
    [1, 0, -d],
    [0, 1, s]
])
A

In [ ]:
WTA= W.inv()@T.inv()@A
WTA.simplify()
WTA

## Generic Omnibot R&S Model

In [ ]:
def generic_omnibot_mats(n = 4, null_beta = True, equal_r = True):
    t, r, d, s = symbols('t r d s')
    alpha, beta= dynamicsymbols('alpha beta')
    
    W = Matrix([
        [r, cos(alpha)],
        [0, sin(alpha)]
    ])
    T = Matrix([
        [cos(beta), -sin(beta)],
        [sin(beta), cos(beta)]
    ])
    A = Matrix([
        [1, 0, -d],
        [0, 1, s]
    ])
    WTA= W.inv()@T.inv()@A
    WTA.simplify()
    r_n = WTA[0,:]
    s_n = WTA[1,:]
    
    R_list = []
    S_list = []
    
    for ii in range(n):
        r_ii = r_n.subs(alpha, symbols('alpha_'+str(ii+1)))
        r_ii = r_ii.subs(d, symbols('d_'+str(ii+1)))
        r_ii = r_ii.subs(s, symbols('s_'+str(ii+1)))
        s_ii = s_n.subs(alpha, symbols('alpha_'+str(ii+1)))
        s_ii = s_ii.subs(d, symbols('d_'+str(ii+1)))
        s_ii = s_ii.subs(s, symbols('s_'+str(ii+1)))
        if not equal_r:
            r_ii = r_ii.subs(r, symbols('r_'+str(ii+1)))
            s_ii = s_ii.subs(r, symbols('r_'+str(ii+1)))
        if null_beta:
            r_ii = r_ii.subs(beta, 0)
            s_ii = s_ii.subs(beta, 0)
        else:
            r_ii = r_ii.subs(beta, symbols('beta_'+str(ii+1)))
            s_ii = s_ii.subs(beta, symbols('beta_'+str(ii+1)))
            
        S_list.append(s_ii)
        R_list.append(r_ii)
    
    R = Matrix(R_list)
    S = Matrix(S_list)
    return R, S

In [ ]:
generic_omnibot_mats(6, null_beta=False )

In [ ]:
def integerize(expr):
    expr2 = expr
    for a in preorder_traversal(expr):
        if isinstance(a, Float):
            dec, _ = modf(a)
            if abs(dec) < 0.0001:
                expr2 = expr2.subs(a, round(a))
    return expr2

In [ ]:
def roundize(expr,n = 4):
    expr2 = expr
    for a in preorder_traversal(expr):
        if isinstance(a, Float):
            expr2 = expr2.subs(a, round(a,n))
    return expr2

### Omnibot with 4 wheels at 45 deg

In [ ]:
def dejabot_mats():
    R, S = generic_omnibot_mats()
    L, l = symbols('L l')
    for ii in range(4):
        alpha = pi/4 * (1 - 2 * ((int((ii+1)/2))%2))
        s = L * (1 - 2 * ((int((ii)/2))%2))
        d = l * (1 - 2 * (ii%2))
        R[ii,:] = R[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        R[ii,:] = R[ii,:].subs(symbols('s_'+str(ii+1)), s)
        R[ii,:] = R[ii,:].subs(symbols('d_'+str(ii+1)), d)
        S[ii,:] = S[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        S[ii,:] = S[ii,:].subs(symbols('s_'+str(ii+1)), s)
        S[ii,:] = S[ii,:].subs(symbols('d_'+str(ii+1)), d)
    return integerize(R), integerize(S)

In [ ]:
R, S = dejabot_mats()
R, S

In [ ]:
R*r

In [ ]:
def omnibot_wheel_mov(R, V = Matrix([symbols('v_1'), symbols('v_2'), dynamicsymbols('psi').diff(t)])):
    if not isinstance(R, Matrix):
        R = Matrix(R)
    if not isinstance(V, Matrix):
        V = Matrix(V)
    return R@V

In [ ]:
omnibot_wheel_mov(R, [1,1,dynamicsymbols('psi').diff(t)])

### Calculating robot coordinates from wheel movement

#### From 3 wheels

In [ ]:
def mu_from_3_wheels(R):
    if not isinstance(R, Matrix):
        R = Matrix(R)
    R_sq = R[:3,:]
    mu = R_sq.inv()
    mu.simplify()
    return mu

In [ ]:
mu = mu_from_3_wheels(R)

In [ ]:
mu

In [ ]:
simplify(R[:3,:]@mu_from_3_wheels(R))

In [ ]:
wheels = omnibot_wheel_mov(R)

In [ ]:
simplify(mu_from_3_wheels(R)@Matrix(wheels[:3]))

#### Least squares method

In [ ]:
def mu_from_least_sq(R):
    return simplify(R.pinv())

In [ ]:
simplify(mu_from_least_sq(R)@wheels)

### 3-wheeled equilateral Omnibot

In [ ]:
def tribot_mats(alpha = pi/2):
    R,S = generic_omnibot_mats(3, null_beta=False)
    l = symbols('l')
    for ii in range(3):
        beta = (ii * pi/3)+pi/2
        s = l * sin(beta)
        d = l * cos(beta)
        R[ii,:] = R[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        R[ii,:] = R[ii,:].subs(symbols('beta_'+str(ii+1)), beta)
        R[ii,:] = R[ii,:].subs(symbols('s_'+str(ii+1)), s)
        R[ii,:] = R[ii,:].subs(symbols('d_'+str(ii+1)), d)
        S[ii,:] = S[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        S[ii,:] = S[ii,:].subs(symbols('beta_'+str(ii+1)), beta)
        S[ii,:] = S[ii,:].subs(symbols('s_'+str(ii+1)), s)
        S[ii,:] = S[ii,:].subs(symbols('d_'+str(ii+1)), d)
    return roundize(R),roundize(S)

In [ ]:
R_3, S_3 = tribot_mats()

In [ ]:
simplify(R_3)*r

In [ ]:
wheels_3 = omnibot_wheel_mov(R_3)
wheels_3

In [ ]:
mu_3_w = mu_from_3_wheels(R_3)
roundize(mu_3_w)

In [ ]:
mu_3_ls = mu_from_least_sq(R_3)
roundize(mu_3_ls)

In [ ]:
integerize(simplify(mu_from_3_wheels(R_3)@Matrix(wheels_3)))

In [ ]:
integerize(simplify(mu_from_least_sq(R_3)@Matrix(wheels_3)))

#### Comparing speed relations forward-lateral

In [ ]:
def simetric_omnibot_mats():
    R, S = generic_omnibot_mats()
    L, l = symbols('L l')
    for ii in range(4):
        alpha = symbols('alpha') * (1 - 2 * ((int((ii+1)/2))%2))
        s = L * (1 - 2 * ((int((ii)/2))%2))
        d = l * (1 - 2 * (ii%2))
        R[ii,:] = R[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        R[ii,:] = R[ii,:].subs(symbols('s_'+str(ii+1)), s)
        R[ii,:] = R[ii,:].subs(symbols('d_'+str(ii+1)), d)
        S[ii,:] = S[ii,:].subs(symbols('alpha_'+str(ii+1)), alpha)
        S[ii,:] = S[ii,:].subs(symbols('s_'+str(ii+1)), s)
        S[ii,:] = S[ii,:].subs(symbols('d_'+str(ii+1)), d)
    return integerize(R), integerize(S)

In [ ]:
R,S = simetric_omnibot_mats()
R,S

In [ ]:
forw_wheels = omnibot_wheel_mov(R, [1,0,0])
lat_wheels = omnibot_wheel_mov(R, [0,1,0])
lat_wheels.simplify()

forw_wheels, lat_wheels

In [ ]:
forw_mov = simplify(mu_from_least_sq(R)@Matrix([1,1,1,1]))
lat_mov  = simplify(mu_from_least_sq(R)@Matrix([-1,1,1,-1]))
forw_mov, lat_mov

In [ ]:
x = np.linspace(0, 80)
y = np.tan(np.pi * x / 180)
plt.figure(figsize=[12,8])
plt.plot(x,y)
plt.title('Lateral distance advance depending on alpha')
plt.xlabel('alpha, in degrees')
plt.ylabel('distance after 1 radian, measured in wheel radius')
plt.grid()


### Looking for the wheel rotation vector that results in no displacement

In [ ]:
R, S = dejabot_mats()
R, S

Obtenemos $\mu$ de los mínimos cuadrados

In [ ]:
min_matr = mu_from_least_sq(R)@Matrix(symbols('theta_1 theta_2 theta_3 theta_4'))
min_matr

This vector must be equal to zero, which gives us these equations:

In [ ]:
v = simplify(min_matr/r*4)
M = Matrix([
    v[0],
    v[1],
    simplify(v[2]*(symbols('L')+symbols('l')))
])
expand(M/symbols('theta_1'))

Si 
$$ A = \theta_2/\theta_1$$

$$B = \theta_3/\theta_1$$

$$C = \theta_4/\theta_1$$

In [ ]:
M = Matrix([
    [1, 1, 1, -1],
    [1, 1, -1, 1],
    [1, -1, 1, 1]
])
M

In [ ]:
M[0,:]+M[1,:]

B = -A

In [ ]:
M[0,:]+M[2,:]

C = -A

Por lo tanto:

$A - 2A = -1$

$A = 1$

Es decir:

$\theta_1 = \theta_2$

$\theta_1 = -\theta_3$

$\theta_1 = -\theta_4$

En otras palabras, cualquier vector de movimiento de las ruedas de forma $x[1,1,-1,-1]$ no produce movimiento:

### Matrix kernel $\mu$ is [1,1,-1,-1]

In [ ]:
a = symbols('a')
mu_from_least_sq(R)@Matrix([a,a,-a,-a])

## Least squares checks

In [ ]:
a,b,c,d,e,f,g,h,i,j,k,l = symbols('a b c d e f g h i j k l')

In [ ]:
Y = Matrix([
    [a,b,c,d],
    [e,f,g,h],
    [i,j,k,l]
])
Y

In [ ]:
D = symbols('D')
R = Matrix([
    [1, -1, -D],
    [1, 1, D],
    [1, 1, -D],
    [1, -1, D]
])
R

In [ ]:
Y@R

Dado que este sistema debe ser $YR = [0]$, obtenemos 3 sistemas de 3 ecuaciones iguales:

a+b+c+d = 0

-a+b+c-d = 0

-a+b-c+d = 0

de(1)+(2):

b+c=0 por lo que b = -c

de (1)+(3):

b+d=0 por lo que b = -d

(1): a + b - b - b = 0

a-b = 0 por lo que a = b

In [ ]:
Y = Matrix([
    [a,a,-a,-a],
    [e,e,-e,-e],
    [i,i,-i,-i]
])
Y

In [ ]:
Y@R

In [ ]:
R

In [ ]:
def diffs_wheels(wheels,R,a,e,i):
    mu = mu_from_least_sq(R)
    Y = Matrix([
        [a,a,-a,-a],
        [e,e,-e,-e],
        [i,i,-i,-i]
    ])
    X = mu + Y
    v = X @ wheels
    ideal_w = R @ v
    return ideal_w - wheels

In [ ]:
error = diffs_wheels(Matrix(symbols('t1 t2 t3 t4')), R, a, e, i)

In [ ]:
error = simplify(error)
E1 = error[0]
E1

In [ ]:
E1_b = collect(E1, symbols('t1 t2 t3 t4'))
E1_b

In [ ]:
E1_b.subs((-D*i+a-e), symbols('z'))

Fila 1: Error = $$-Di+a-e-1/4$$

In [ ]:
error[1].collect(symbols('t1 t2 t3 t4'))

In [ ]:
error[2].collect(symbols('t1 t2 t3 t4'))

In [ ]:
error[3].collect(symbols('t1 t2 t3 t4'))

$$-Di+a-e-1/4$$
$$Di+a+e-1/4$$
$$-Di+a+e+1/4$$
$$Di+a-e+1/4$$

In [ ]:
tot_err = ((-D*i+a-e-1/4)**2 + 
           (D*i+a+e-1/4)**2 + 
           (-D*i+a+e+1/4)**2 + 
           (D*i+a-e+1/4)**2)
tot_err = simplify(tot_err)
tot_err

### Alternative formulation which lands us at the same result:

In [ ]:
X = Y+mu_from_least_sq(R)
X

In [ ]:
simplify((R@X-eye(4))@Matrix(symbols('t1 t2 t3 t4')))

# Euler-Lagrange Dynamics

In [ ]:
m, I_z, I_w =symbols('m I_z I_w')

In [ ]:
x, y, psi = dynamicsymbols('x y psi')
q = [x, y, psi] + [dynamicsymbols(f'phi_{i+1}') for i in range(4)]
q

In [ ]:
M_w = eye(4)*I_w
M_w

In [ ]:
M_r = Matrix([
    [m, 0, 0],
    [0, m, 0],
    [0, 0, I_z]
])
M_r

In [ ]:
Gamma = [dynamicsymbols(f'tau_{i+1}') for i in range(4)]
Gamma = Matrix(Gamma)
Gamma

In [ ]:
R_psi = Matrix([
    [cos(psi), -sin(psi), 0],
    [sin(psi), cos(psi), 0],
    [0, 0, 1]
])
R_psi

In [ ]:
R, S = dejabot_mats()
R

In [ ]:
H = M_r + R_psi@R.T@M_w@R@R_psi.T
H = simplify(H)
H

In [ ]:
K = R_psi@R.T@M_w@R@R_psi.diff().T
K = simplify(K)
K

In [ ]:
F_a = R_psi@R.T@Gamma
F_a

In [ ]:
F_a_0 = F_a[0].factor(sin(psi), cos(psi))
F_a_0

In [ ]:
F_a_1 = F_a[1].factor(sin(psi), cos(psi))
F_a_1

In [ ]:
L, l = symbols('L l')
F_a_2 = F_a[2].factor()
F_a_2

In [ ]:
F_a = Matrix([F_a_0, F_a_1, F_a_2])
F_a

In [ ]:
A = R_psi@R.T
A

In [ ]:
B = A.pinv()
B = simplify(B)
B

In [ ]:
simplify(H.inv())